In [ ]:
from tqdm.notebook import trange
import chipwhisperer as cw
import os
import numpy as np
import time
import csv

In [ ]:
PLATFORM = 'CW308_STM32F3'
CRYPTO_TARGET = 'NONE'
SCOPETYPE = 'OPENADC'
SS_VER = 'SS_VER_1_1'


In [ ]:
scope = cw.scope()
scope.default_setup()

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../firmware/mcu/simpleserial-ascon
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

In [ ]:
%run "Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = "../firmware/mcu/simpleserial-ascon/simpleserial-ascon-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)

In [ ]:
if PLATFORM == "CWLITEXMEGA":
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        target.flush()
else:
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = "high_z"
        time.sleep(0.05)
        target.flush()

In [ ]:
reboot_flush()

try:
    target.baud = BAUD_RATE
except:
    pass

print('Baudrate:')
print(target.baud)


In [ ]:
import numpy as np
import time

CMD_ASCON = 'a'

DATA_LEN = 190
RESP_LEN = 96

M = 0x01             
C = 0x02              
A = 0x04              
N = 0x08              
K = 0x10              
RUN_ENC = 0x20          
OMIT_RESULT = 0x80   

def pad(data):
    if len(data) > DATA_LEN:
        print("data length: " + str(len(data)))
    assert len(data) <= DATA_LEN
    if SS_VER == 'SS_VER_1_1':
        data += bytearray.fromhex('00' * (DATA_LEN - len(data)))
    return data

m = bytearray.fromhex('00000000000000000000000000000000') 
a = bytearray.fromhex('00')  
n = bytearray.fromhex('00000000000000000000000000000000')  
k = bytearray.fromhex('00000000000000000000000000000000') 


data = bytearray([N | K])
data += n  
data += k  
target.flush()
target.simpleserial_write(CMD_ASCON, pad(data))
time.sleep(0.1)

data = bytearray([M | A | RUN_ENC])
data += bytearray([len(m)]) + m  # Message
data += bytearray([len(a)]) + a  # Associated data

scope.arm()
target.flush()
target.simpleserial_write(CMD_ASCON, pad(data))
time.sleep(0.1)

ret = scope.capture()
if ret:
    print("Capture Timed Out!")
    reboot_flush()
else:
    trace = scope.get_last_trace()

response = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)
print(response)

mlen = len(m)
tlen = 40  # Updated tag length from 16 to 40 bytes

c = response[0:mlen]  
t = response[mlen:mlen + tlen] 

print('Ciphertext: ' + ''.join('{:02x}'.format(x) for x in c))
print('Tag: ' + ''.join('{:02x}'.format(x) for x in t))

print(trace)


In [ ]:
import logging
from tqdm.notebook import trange
import numpy as np
import time

CMD_ASCON1 = 'a'


# Flags
M1 = 0x01             
C1 = 0x02             
A1 = 0x04             
N1 = 0x08             
K1 = 0x10             
RUN_ENC1 = 0x20      
OMIT_RESULT1 = 0x80   


def pad(data):
    if len(data) > DATA_LEN:
        print("data length: " + str(len(data)))
    assert len(data) <= DATA_LEN
    if SS_VER == 'SS_VER_1_1':
        data += bytearray.fromhex('00' * (DATA_LEN - len(data)))
    return data

# Define input data
m1 = bytearray.fromhex('00000000000000000000000000000000')  
a1 = bytearray.fromhex('00')  
n1 = bytearray.fromhex('00000000000000000000000000000000')  
k1 = bytearray.fromhex('00000000000000000000000000000000')  

# Send nonce and key
data1 = bytearray([N1 | K1])
data1 += n1  
data1 += k1  
target.flush()
target.simpleserial_write(CMD_ASCON1, pad(data1))
time.sleep(0.1)

# Send plaintext and associated data
data1 = bytearray([M1 | A1 | RUN_ENC1])
data1 += bytearray([len(m1)]) + m1  
data1 += bytearray([len(a1)]) + a1  

scope.arm()
target.flush()
target.simpleserial_write(CMD_ASCON1, pad(data1))
time.sleep(0.1)

ret = scope.capture()
if ret:
    print("Capture Timed Out!")
    reboot_flush()
else:
    trace = scope.get_last_trace()

response1 = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)
print(response1)

mlen1 = len(m1)
tlen1 = 40  
clen1 = mlen1 + tlen1  

c1 = response1[0:mlen1] 
t1 = response1[mlen1:mlen1 + tlen1]  

print('Ciphertext1: ' + ''.join('{:02x}'.format(x) for x in c1))
print('Tag1: ' + ''.join('{:02x}'.format(x) for x in t1))


In [ ]:
results = []
traces = [] 

CMD_ASCON11 = 'a'

# Flags
M11 = 0x01             
C11 = 0x02             
A11 = 0x04              
N11 = 0x08             
K11 = 0x10             
RUN_ENC11 = 0x20       
# RUN_DEC11 = 0x40     
OMIT_RESULT11 = 0x80   

def pad(data):
    if len(data) > DATA_LEN:
        print("data length: " + str(len(data)))
    assert len(data) <= DATA_LEN
    if SS_VER == 'SS_VER_1_1':
        data += bytearray.fromhex('00' * (DATA_LEN - len(data)))
    return data

m11 = bytearray.fromhex('00000000000000000000000000000000')  
a11 = bytearray.fromhex('00')  
n11 = bytearray.fromhex('00000000000000000000000000000000')  
k11 = bytearray.fromhex('00000000000000000000000000000000')  

data11 = bytearray([N11 | K11])
data11 += n11  
data11 += k11  
target.flush()
target.simpleserial_write(CMD_ASCON11, pad(data11))
time.sleep(0.1)

data11 = bytearray([M11 | A11 | RUN_ENC11])
data11 += bytearray([len(m11)]) + m11 
data11 += bytearray([len(a11)]) + a11 

scope.arm()
target.flush()
target.simpleserial_write(CMD_ASCON11, pad(data11))
time.sleep(0.1)

ret1 = scope.capture()
if ret1:
    print("Capture Timed Out!")
    reboot_flush()
else:
    trace = scope.get_last_trace()

response11 = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)
print(response11)

# print the response
mlen11 = len(m11)
tlen11 = 40  # Updated tag length from 16 to 40 bytes

c11 = response11[0:mlen11]  
t11 = response11[mlen11:mlen11+tlen11]

print('Ciphertext11: ' + ''.join('{:02x}'.format(x) for x in c11))
print('Tag: ' + ''.join('{:02x}'. format(x) for x in t11))


In [ ]:
if scope._is_husky:
    scope.glitch.enabled = True
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"
scope.glitch.repeat = 1
scope.io.hs2 = "glitch"
scope.glitch.width = 2
scope.glitch.offset = -4.0
scope.glitch.ext_offset=0
print(scope.glitch)
print(scope)

In [ ]:
print(scope.clock)

In [ ]:
scope.clock.adc_src = "clkgen_x1"

scope.arm()
print(scope)

In [ ]:
print(scope.clock)

In [ ]:
import time
import logging
import numpy as np
from tqdm.notebook import trange
      
CMD_ASCON111 = 'a'

M111 = 0x01              # data contains message 
C111 = 0x02              # data contains ciphertext
A111 = 0x04              # data contains associated data
N111 = 0x08              # data contains nonce
K111 = 0x10              # data contains key
RUN_ENC111 = 0x20        # run enc after data transmission
# RUN_DEC111 = 0x40        # run dec after data transmission
OMIT_RESULT111 = 0x80    # omit returning result after encryption/decryp

logging.getLogger().setLevel(logging.ERROR)

num_tests = 1000 
glitch_loc = range(121, 124) 
results = []
traces = []

for test in range(num_tests):
    reboot_flush()
    time.sleep(0.1)

    for i in trange(min(glitch_loc), max(glitch_loc) + 1, desc=f"Glitch Test {test+1}"):
        scope.adc.timeout = 0.2
        scope.glitch.ext_offset = i  
        scope.arm()

        
        m111 = bytearray.fromhex('00000000000000000000000000000000')  
        a111 = bytearray.fromhex('00')  
        n111 = bytearray.fromhex('00000000000000000000000000000000')  
        k111 = bytearray.fromhex('00000000000000000000000000000000')  

        
        data111 = bytearray([N111 | K111])
        data111 += n111  
        data111 += k111  
        target.flush()
        target.simpleserial_write(CMD_ASCON11, pad(data111))
        time.sleep(0.1)

        
        data111 = bytearray([M111 | A111 | RUN_ENC111])
        data111 += bytearray([len(m111)]) + m111  
        data111 += bytearray([len(a111)]) + a111  

        scope.arm()
        target.flush()
        target.simpleserial_write(CMD_ASCON111, pad(data111))
        time.sleep(0.1)

        ret1 = scope.capture()
        if ret1:
            print("⚠️ Capture Timed Out! Resetting...")
            reboot_flush()
            continue

        trace = scope.get_last_trace()

        
        glitched_response111 = target.simpleserial_read_witherrors('r', RESP_LEN, end='\n', timeout=250, ack=True)

        if glitched_response111['valid']:
            glitched_t111 = glitched_response111['payload'][mlen11:mlen11+tlen11]  

           
            if glitched_t111 != t11:
                print(f"\n Glitch at {i}:")
                print(f"   Original tag: {t11.hex()}") 
                print(f"   Glitched tag: {glitched_t111.hex()}")
                
                break  
        else:
            reboot_flush()
